# Worksheet 07

Name: Shivangi
UID: U35642613

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[4.364736777835721, 4.142142303238222, 4.904372236371267, 5.995438076678698, 3.7336059807255064, 4.566764387506843, 6.212205935971271, 4.226828072827067, 4.7504478694790695, 4.408188508044759]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
c2 = np.random.normal(8, 1, 10).tolist()
print(c2)

[7.637599186894538, 9.658829466846022, 8.55394551327187, 7.454787785669766, 6.784462657941731, 8.882138684400621, 7.91909287826638, 7.348539400792135, 8.667906931409279, 8.619104709369406]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[8.619104709369406, 8.667906931409279, 4.408188508044759, 4.7504478694790695, 7.348539400792135, 7.91909287826638, 4.226828072827067, 6.212205935971271, 4.566764387506843, 8.882138684400621]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

c1 = [6.201965597819907, 5.9833374131375034, 5.371864977398792, 3.865299608808478, 6.186975859415653, 6.654768094026854, 4.210830965466652, 3.2130158785226683, 3.5613384874433383, 8.357196861603997]

c2 = [7.914524653792445, 9.747176655148124, 9.05984533837469, 7.208736703051519, 6.685070547267801, 8.166563875054502, 7.578130450948437, 7.270265942252881, 9.490677786351522, 7.564693234490808]

data = [7.564693234490808, 8.357196861603997, 9.490677786351522, 7.270265942252881, 7.578130450948437, 3.5613384874433383, 3.2130158785226683, 8.166563875054502, 6.685070547267801, 4.210830965466652]

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [12]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2)) ]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[3.5613384874433383, 3.2130158785226683, 4.210830965466652]
[7.564693234490808, 8.357196861603997, 9.490677786351522, 7.270265942252881, 7.578130450948437, 8.166563875054502, 6.685070547267801]
P(C_1) = 0.3,  P(C_2) = 0.7
mean_1 = 3.6617284438108864,  mean_2 = 7.8732283854242775
var_1 = 0.17097822962524403,  var_2 = 0.6991390352162233


e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [13]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append(pdf_i[0] * prob_c[0]/ prob_x)
    prob_c1_x.append(pdf_i[1] * prob_c[1]/ prob_x)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  7.564693234490808
probability of observing that point if it came from cluster 0 =  1.644535414953089e-113
probability of observing that point if it came from cluster 1 =  0.5176743287667553
point =  8.357196861603997
probability of observing that point if it came from cluster 0 =  3.975656547383265e-164
probability of observing that point if it came from cluster 1 =  0.4490471001412495
point =  9.490677786351522
probability of observing that point if it came from cluster 0 =  9.74880633102309e-253
probability of observing that point if it came from cluster 1 =  0.039275866607428406
point =  7.270265942252881
probability of observing that point if it came from cluster 0 =  4.403247431616752e-97
probability of observing that point if it came from cluster 1 =  0.39339915296847433
point =  7.578130450948437
probability of observing that point if it came from cluster 0 =  2.726327090779439e-114
probability of observing that point if it came from cluster 1 =  0.5219873819572628
poin

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [22]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/len(prob_c1_x) ]
mean = [np.mean(prob_c0_x), np.mean(prob_c1_x)]
var = [np.var(prob_c0_x), np.var(prob_c1_x)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.29998910584477917,  P(C_2) = 0.7000108941552208
mean_1 = 0.29998910584477917,  mean_2 = 0.7000108941552208
var_1 = 0.20998474925066568,  var_2 = 0.20998474925066563


These updated values are different from the original ones obtained from K-means because they are based on the probability distribution of the data, rather than just the cluster assignments. The updated mean_j values represent the centers of the clusters in the Gaussian mixture model, while the updated var_j values represent the spread of the data in each cluster. The updated P(C_j) values represent the prior probabilities of each cluster.

g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [27]:
prob_c0_x = []
prob_c1_x = []

for p in data:
    pdf_i = [norm.pdf(p, mean[j], var[j]) for j in range(k)]
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]
    # prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x)
    # prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x)
    if prob_x == 0:
        prob_c0_x.append(0)
        prob_c1_x.append(0)
    else:
        prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x)
        prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x)

print(prob_c0_x, prob_c1_x)



[6.271205910495445e-29, 4.730361634339807e-32, 0, 9.06534879019313e-28, 5.551481382100406e-29, 3.719048041308592e-13, 8.766191418277058e-12, 2.6668668320515484e-31, 1.832459360943049e-25, 1.0267238431591208e-15] [1.0, 1.0, 0, 1.0, 1.0, 0.9999999999996281, 0.9999999999912338, 1.0, 1.0, 0.999999999999999]


The difference between the original K-means estimates and the updated estimates obtained from soft clustering indicates how well soft clustering algorithm works. If the difference is small, it means that soft clustering is converging towards the correct solution and the estimates are improving. On the other hand, if the difference is large, it may indicate that the algorithm is not converging, or that it is converging towards a suboptimal solution.

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [29]:
if(prob_c0_x > prob_c1_x):
    print("Cluster 0")
else:
    print("Cluster 1")

Cluster 1


### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      1       |
| A  C |      0       |
| A  D |      0       |
| A  E |      0       |
| B  C |      1       |
| B  D |      0       |
| B  E |      0       |
| C  D |      0       |
| C  E |      0       |
| D  E |      1       |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

N(N-1)/2

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

There are 9 points in the dataset, maximum number of disagreements is 36

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

For cluster 0, there are (3 x 2) / 2 = 3 pairs that agree with C.

For cluster 1, there are (2 x 1) / 2 = 1 pair that agrees with C.

For cluster 2, there are (4 x 3) / 2 = 6 pairs that agree with C.

The total number of agreements is 3 + 1 + 6 = 10.

Since there are 9 points in the dataset, and we have 10 agreements, there must be 9 - 10 = -1 disagreements between C and P. However, a negative number of disagreements doesn't make sense in this context. So we can say that there are actually 0 disagreements between C and P, since all the points in C are in the same cluster in P.

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.

Algo:
1. Compute the clusters for the dataset using a clustering algorithm (e.g. k-means or EM).
2. For each cluster, identify all pairs of points within that cluster and count the number of pairs that agree with the clustering result (i.e., the number of pairs that are assigned to the same cluster).
3. Compute the total number of agreements by summing up the number of agreements for each cluster.
4. Compute the total number of disagreements by subtracting the total number of agreements from the total number of point pairs in the dataset (which can be computed using the formula from part (b) of the question).
5. Repeat the process for any other clustering results that need to be compared with the original clustering.

This algorithm has a time complexity of O(N^2) for step 2, since it requires examining all pairs of points within each cluster. However, if the number of clusters is much smaller than the total number of points (e.g. if the clustering result is very good), then this algorithm can be quite efficient. Additionally, steps 3-5 can be computed in O(1) time once the number of agreements and disagreements have been computed, so the overall time complexity is still quite reasonable.